# Ciekawostki z [Biblioteki Standardowej Pythona](https://docs.python.org/3/library/#the-python-standard-library)

## [`json`](https://docs.python.org/3/library/json.html#module-json) do kodowania i dekodowania JSON
Ponieważ internet jest obecnie wypełniony JSON-em, a dobre czasy xml-a minęły.

In [ ]:
data = {"b": True, "a": 1, "nested": {"foo": "bar"}, "c": None, "some_list": [1, 2, 3]}

### Kodowanie

In [ ]:
import json

json_data = json.dumps(data)
print(f"type: {type(json_data)} data: {json_data}")

### Dekodowanie

In [ ]:
decoded = json.loads(json_data)
print(f"type: {type(decoded)} data: {decoded}")

## [`unittest.mock`](https://docs.python.org/3/library/unittest.mock.html#module-unittest.mock)
Chociaż `pytest` jest preferowanym frameworkiem do testowania, moduł `unittest.mock` oferuje kilka przydatnych rzeczy, które są pomocne również w przypadkach testowych `pytest`. Mockowanie i patchowanie są ogólnie przydatne do "udawania" niektórych części logiki/stanu oprogramowania poddawanego testom. Typowe przypadki użycia to na przykład patchowanie fragmentów kodu, które wchodzą w interakcję z osobami trzecimi (np. niektórymi usługami internetowymi).

### [`MagicMock`](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.MagicMock)

Ogólnie rzecz biorąc, [Mocki](https://en.wikipedia.org/wiki/Mock_object) to symulowane obiekty, które zastępują funkcjonalność/stan obiektu ze świata rzeczywistego w kontrolowany sposób. Dlatego są szczególnie przydatne w testach do naśladowania pewnego zachowania określonej części implementacji poddawanej testom.

W standardowej bibliotece Pythona istnieje również klasa [`Mock`](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.Mock), ale zazwyczaj chcesz używać [`MagicMock`](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.MagicMock), która jest podklasą `Mock`. `MagicMock` zapewnia domyślną implementację większości metod magicznych (np. `__setitem__()` i `__getitem__()`).

Potencjalny przypadek użycia może wyglądać następująco:

In [ ]:
import random


class Client:
    def __init__(self, url, username, password):
        self.url = url
        self.creds = (username, password)

    def fetch_some_data(self):
        print(
            "Here we could for example fetch data from 3rd party API and return the data."
        )
        print("Now we will just return some random number between 1-100.")
        return random.randint(1, 100)


class MyApplication:
    def __init__(self):
        self.client = Client(
            url="https://somewhere/api", username="John Doe", password="secret123?"
        )

    def do_something_fancy(self):
        data = self.client.fetch_some_data()
        return data ** (1 / 2)  # let's return a square root just for example


####################
# In the test module:

from unittest.mock import MagicMock

# Inside a test case:
app = MyApplication()
app.client = MagicMock()  # Mock the client
app.client.fetch_some_data.return_value = 4  # Set controlled behaviour
result = app.do_something_fancy()
assert result == 2
print("All good, woop woop!")

### [`patch`](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.patch)
Przypadki użycia [`patch`](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.patch) są dość podobne do `MagicMock`. Największą różnicą jest to, że `patch` jest używany jako menedżer kontekstu lub dekorator. Obiekt do załatania jest podawany jako argument do `patch`. Dodatkowo można podać dodatkowy obiekt jako drugi argument (`new`), który zastąpi oryginalny. W przypadku pominięcia `new`, domyślnie zostanie użyty `MagicMock`.

Zobaczmy, jak powyższy przykład wyglądałby z `patch`.

In [ ]:
# In the test module:

from unittest.mock import patch

# Inside a test case:
app = MyApplication()
with patch("__main__.app.client") as patched_client:  # Patch the client
    patched_client.fetch_some_data.return_value = 4  # Set controlled behaviour
    result = app.do_something_fancy()
    assert result == 2
    print("All good, woop woop!")

To samo, ale z dekoratorem funkcji zamiast menedżera kontekstu. Zauważ, że tutaj łatamy całą klasę `Client`, a nie tylko zmienną instancji `client` w `app`.

In [ ]:
from unittest.mock import patch


@patch("__main__.Client")  # Patch the Client
def test_do_something_fancy(client_cls):
    client_cls().fetch_some_data.return_value = 4  # Set controlled behaviour
    app = MyApplication()
    result = app.do_something_fancy()
    assert result == 2
    print("All good, woop woop!")


test_do_something_fancy()  # This is just for the sake of example

## [`collections`](https://docs.python.org/3/library/collections.html#module-collections)

### [`namedtuple`](https://docs.python.org/3/library/collections.html#collections.namedtuple)
Świetny pomocnik do tworzenia bardziej czytelnego i samodokumentującego się kodu.

`namedtuple` to funkcja, która zwraca krotkę, której pola mają nazwy, a także sama krotka ma nazwę (podobnie jak klasy i ich zmienne instancji). Potencjalne przypadki użycia obejmują przechowywanie danych, które powinny być niezmienne. Jeśli możesz używać Pythona 3.7 lub nowszego, możesz również rzucić okiem na [`dataclasses`](https://docs.python.org/3/library/dataclasses.html#module-dataclasses).

In [ ]:
from collections import namedtuple

Person = namedtuple("Person", ["name", "age", "is_gangster"])

# instance creation is similar to classes
john = Person("John Doe", 83, True)
lisa = Person("Lis Doe", age=77, is_gangster=False)

print(john, lisa)
print(f"Is John a gangster: {john.is_gangster}")

# tuples are immutable, thus you can't do this
# john.is_gangster = False

### [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter)
Do zliczania wystąpień elementów w kolekcji.

In [ ]:
from collections import Counter

data = [1, 2, 3, 1, 2, 4, 5, 6, 2]

counter = Counter(data)
print(f"type: {type(counter)}, counter: {counter}")

print(f"count of twos: {counter[2]}")
print(f"count of tens: {counter[10]}")  # zero for non existing

print(f"counter is a dict: {isinstance(counter, dict)}")

### [`defaultdict`](https://docs.python.org/3/library/collections.html#collections.defaultdict)
Dla czystszego kodu do wypełniania słowników.

Najpierw zobaczmy, jak można użyć zwykłego `dict`.

In [ ]:
data = (1, 2, 3, 4, 3, 2, 5, 6, 7)

my_dict = {}
for val in data:
    if val % 2:
        if not "odd" in my_dict:
            my_dict["odd"] = []
        my_dict["odd"].append(val)
    else:
        if not "even" in my_dict:
            my_dict["even"] = []
        my_dict["even"].append(val)

print(my_dict)

Z `defaultdict`:

In [ ]:
from collections import defaultdict

my_dict = defaultdict(list)
for val in data:
    if val % 2:
        my_dict["odd"].append(val)
    else:
        my_dict["even"].append(val)
print(my_dict)

W powyższym przykładzie `defaultdict` zapewnia, że nowa `lista` jest automatycznie inicjowana jako wartość po dodaniu nowego klucza.

Oto kolejny przykład z `int` jako wartością domyślną.

In [ ]:
my_dict = defaultdict(int)
for val in data:
    if val % 2:
        my_dict["odd_count"] += 1
    else:
        my_dict["even_count"] += 1
print(my_dict)